In [1]:
import sys
sys.path.append("..")

In [2]:
from train_autoencoder import *
from test_autoencoder import *

In [3]:
from C2D_Models import *
from PatchWise.models_PatchWise import *

In [4]:
DATA_TYPE = "ucsd2"
IMAGE_TYPE = "normal" # change
OPTIMIZER_TYPE = "adam"
LOSS_TYPE = "mse"
BATCH_SIZE = 64
DENOISING = False
IMAGE_SIZE = 128
EPOCHS = 300
EXTRA_TAG = "APE"

In [5]:
train_data, CHANNELS = select_dataset(dataset=DATA_TYPE, isTrain=True, image_size = IMAGE_SIZE, image_type = IMAGE_TYPE)

models = [
    C2D_AE_128_3x3(channels = CHANNELS),
    C2D_AE_128_3x3_SQZEX(channels = CHANNELS),
    ConvAttentionWrapper(C2D_AE_128_3x3(channels = CHANNELS)),
    SoftMaxConvAttentionWrapper(C2D_AE_128_3x3(channels = CHANNELS)),
]

LOSS_TYPES = [LOSS_TYPE] * len(models)
# LOSS_TYPES = ["mse", "weighted", "weighted", "weighted"]
OPTIMIZERS_TYPES = [OPTIMIZER_TYPE] * len(models)

model_files = [
        complete_model_name(
            m.__name__,
            optimizer_type=OPTIMIZER_TYPE,
            loss_type=LOSS_TYPE,
            dataset_type=DATA_TYPE,
            image_type=IMAGE_TYPE,
            isDeNoising=DENOISING,
            extra = EXTRA_TAG
        ) for m, LOSS_TYPE, OPTIMIZERS_TYPE in zip(models, LOSS_TYPES, OPTIMIZERS_TYPES)
    ]

model_files = [join_paths(["../../ATTN_PPR_MODELS/"+DATA_TYPE.upper()+"/", mn]) for mn in model_files]
pprint(model_files)

16it [00:02,  7.36it/s]

['../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3_SQZEX-_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_CONV_ATTENTION_P64_L1E-06_C1_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_SOFTMAX_ATTENTION_P64_C1_ADAM_MSE_UCSD2_NORMAL-{APE}']


# Training

In [6]:
optimizers = [select_optimizer[ot](m) for m,ot in zip(models, OPTIMIZERS_TYPES)]
losses = [select_loss[lt] for m,lt in zip(models, LOSS_TYPES)]

In [7]:
train_loader, val_loader = get_data_loaders(train_data, batch_size=BATCH_SIZE)

In [8]:
trainer = AutoEncoder_Trainer(
    models,
    model_files,
    train_loader, 
    val_loader,
    optimizers,
    losses,
    epochs = EPOCHS
)

In [9]:
trainer.train()

Model: ../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}.pth.tar
Maximum estimated Time: 2.40 hours | Will be completed by: 2021-02-11 16:40
------------------------------------------------------------

/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


/usr/bin/nvidia-modprobe: unrecognized option: "-s"

ERROR: Invalid commandline, please run `/usr/bin/nvidia-modprobe --help` for usage information.


Thu Feb 11 14:15:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.102.04   Driver Version: 450.102.04   CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M.

['../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3_SQZEX-_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_CONV_ATTENTION_P64_L1E-06_C1_ADAM_MSE_UCSD2_NORMAL-{APE}',
 '../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_SOFTMAX_ATTENTION_P64_C1_ADAM_MSE_UCSD2_NORMAL-{APE}']

In [10]:
print("Training done")

Training done


# Testing

In [11]:
test_data, CHANNELS = select_dataset(dataset = DATA_TYPE, image_size = IMAGE_SIZE, image_type = IMAGE_TYPE, isTrain=False, sample_stride = 1)

../general/data.py:486: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  temporal_labels = np.array(temporal_labels)
12it [00:02,  4.10it/s]


In [12]:
for ae_model in trainer.autoencoder_models:
    print("-"*40)
    print(ae_model.model_file)
    ae_model.model.isTrain = False
    tester = AutoEncoder_Tester(
        model = ae_model.model,
        dataset = test_data,
        model_file = ae_model.model_file,
        stackFrames = 64,
        save_vis = False,
        useGPU = True
    )
    results = tester.test()
    print("="*40)
    del tester

0it [00:00, ?it/s]

----------------------------------------
../../ATTN_PPR_MODELS/UCSD2/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}/C2D_AE_128_3X3-_ADAM_MSE_UCSD2_NORMAL-{APE}.pth.tar


7it [00:05,  1.37it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
8it [00:05,  1.36it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/m

OrderedDict([('auc_roc_score_agg_absolute', 0.8739138014268089), ('auc_roc_score_agg_psnr', 0.8453256584240735), ('auc_roc_score_agg_squared', 0.8384966877648448), ('auc_roc_score_agg_ssim', 0.8261847610363139), ('auc_roc_score_mean_absolute', 0.924842366596246), ('auc_roc_score_mean_psnr', 0.8266458877419519), ('auc_roc_score_mean_squared', 0.8259738667521), ('auc_roc_score_mean_ssim', 0.8718140138456316), ('auc_roc_score_overall_absolute', 0.6949558815641259), ('auc_roc_score_overall_psnr', 0.7255420935471759), ('auc_roc_score_overall_squared', 0.723803840583597), ('auc_roc_score_overall_ssim', 0.7107241994314221), ('eer_agg_absolute', 0.19889502762430938), ('eer_agg_psnr', 0.25138121546961234), ('eer_agg_squared', 0.2651933701657435), ('eer_agg_ssim', 0.2659107534747623), ('eer_overall_absolute', 0.4538703870387114), ('eer_overall_psnr', 0.3816747572816059), ('eer_overall_squared', 0.3828883495133143), ('eer_overall_ssim', 0.4203730272596844), ('pr_roc_score_agg_absolute', 0.6584254

7it [00:05,  1.38it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
8it [00:05,  1.38it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/m

OrderedDict([('auc_roc_score_agg_absolute', 0.8578303518746981), ('auc_roc_score_agg_psnr', 0.8317029179853028), ('auc_roc_score_agg_squared', 0.8270513731695542), ('auc_roc_score_agg_ssim', 0.8242487126535429), ('auc_roc_score_mean_absolute', 0.9092372698471153), ('auc_roc_score_mean_psnr', 0.8528057022550501), ('auc_roc_score_mean_squared', 0.8535654849785606), ('auc_roc_score_mean_ssim', 0.8790294843253345), ('auc_roc_score_overall_absolute', 0.6988346832591321), ('auc_roc_score_overall_psnr', 0.7347412567719788), ('auc_roc_score_overall_squared', 0.7335159309124067), ('auc_roc_score_overall_ssim', 0.7240133696293515), ('eer_agg_absolute', 0.22087378640752164), ('eer_agg_psnr', 0.2578883495144584), ('eer_agg_squared', 0.2718446601946132), ('eer_agg_ssim', 0.28033980582534973), ('eer_overall_absolute', 0.4168689320385596), ('eer_overall_psnr', 0.3539618276460382), ('eer_overall_squared', 0.3563535911602211), ('eer_overall_ssim', 0.38531553398150575), ('pr_roc_score_agg_absolute', 0.6

7it [00:05,  1.25it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
8it [00:06,  1.24it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/m

OrderedDict([('auc_roc_score_agg_absolute', 0.8789978141929947), ('auc_roc_score_agg_psnr', 0.8654873813227484), ('auc_roc_score_agg_squared', 0.8688783993992384), ('auc_roc_score_agg_ssim', 0.8626126428149976), ('auc_roc_score_mean_absolute', 0.9249344771923423), ('auc_roc_score_mean_psnr', 0.8812964315867792), ('auc_roc_score_mean_squared', 0.8816421447782619), ('auc_roc_score_mean_ssim', 0.926183252613371), ('auc_roc_score_overall_absolute', 0.6998890334173684), ('auc_roc_score_overall_psnr', 0.7290571528187524), ('auc_roc_score_overall_squared', 0.7261270986429222), ('auc_roc_score_overall_ssim', 0.742404991149493), ('eer_agg_absolute', 0.19613259668508345), ('eer_agg_psnr', 0.19435075885328834), ('eer_agg_squared', 0.19356796116592528), ('eer_agg_ssim', 0.21270718232053978), ('eer_overall_absolute', 0.4346358792184715), ('eer_overall_psnr', 0.3762135922326505), ('eer_overall_squared', 0.3737864077677694), ('eer_overall_ssim', 0.38059701492570497), ('pr_roc_score_agg_absolute', 0.6

7it [00:05,  1.25it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
8it [00:06,  1.24it/s]/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/metrics/_ranking.py:681: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/ambreesh/Documents/amb_venv/lib/python3.6/site-packages/sklearn/m

OrderedDict([('auc_roc_score_agg_absolute', 0.8769477820093332), ('auc_roc_score_agg_psnr', 0.8809556535965241), ('auc_roc_score_agg_squared', 0.8805717963847021), ('auc_roc_score_agg_ssim', 0.7664136673282197), ('auc_roc_score_mean_absolute', 0.890881460173721), ('auc_roc_score_mean_psnr', 0.8958737227812197), ('auc_roc_score_mean_squared', 0.8957816431110558), ('auc_roc_score_mean_ssim', 0.7951458491221577), ('auc_roc_score_overall_absolute', 0.6960504612991472), ('auc_roc_score_overall_psnr', 0.6834803947862468), ('auc_roc_score_overall_squared', 0.6834686611596845), ('auc_roc_score_overall_ssim', 0.6064306978490586), ('eer_agg_absolute', 0.19660194174755558), ('eer_agg_psnr', 0.1878453038674025), ('eer_agg_squared', 0.18784530386740322), ('eer_agg_ssim', 0.35922330097174987), ('eer_overall_absolute', 0.3502321981424181), ('eer_overall_psnr', 0.37016574585635986), ('eer_overall_squared', 0.3701657458563689), ('eer_overall_ssim', 0.4558011049723757), ('pr_roc_score_agg_absolute', 0.6

In [13]:
exit()